## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-17-50-00 +0000,wsj,Opinion | DOGE Wreaks Havoc on Education Research,https://www.wsj.com/opinion/doge-wreaks-havoc-...
1,2025-09-05-17-49-48 +0000,nyt,What We Know About Trump’s Crime and Immigrati...,https://www.nytimes.com/article/trump-national...
2,2025-09-05-17-49-27 +0000,nyt,"After UK Deputy Premier Resigns, Farage Seeks ...",https://www.nytimes.com/2025/09/05/world/europ...
3,2025-09-05-17-49-00 +0000,wsj,Opinion | Make GPAs Intelligible Again,https://www.wsj.com/opinion/make-gpas-intellig...
4,2025-09-05-17-48-00 +0000,wsj,Opinion | Can Maduro Stand the Heat?,https://www.wsj.com/opinion/can-maduro-stand-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,62
85,up,13
578,department,13
27,new,13
103,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
262,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...,130
256,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,127
374,2025-09-04-18-19-18 +0000,bbc,Trump's Fed pick says he will keep White House...,https://www.bbc.com/news/articles/c5yq3z2xd01o...,112
213,2025-09-05-06-52-10 +0000,nypost,"Trump’s ‘pro-business, pro-innovation’ agenda ...",https://nypost.com/2025/09/05/us-news/trumps-p...,110
241,2025-09-05-02-44-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
262,130,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...
72,67,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
28,62,2025-09-05-17-04-18 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
256,59,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...
374,50,2025-09-04-18-19-18 +0000,bbc,Trump's Fed pick says he will keep White House...,https://www.bbc.com/news/articles/c5yq3z2xd01o...
210,47,2025-09-05-07-53-33 +0000,wsj,Stock Market Today: Weak Jobs Data Boosts Rate...,https://www.wsj.com/livecoverage/jobs-report-a...
179,46,2025-09-05-09-42-52 +0000,nypost,Putin warns Western troops deployed to Ukraine...,https://nypost.com/2025/09/05/world-news/russi...
81,45,2025-09-05-15-21-27 +0000,nyt,Israel Steps Up Attacks on Gaza City Ahead of ...,https://www.nytimes.com/2025/09/05/world/middl...
319,44,2025-09-04-21-25-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
74,40,2025-09-05-15-39-10 +0000,nyt,"Angela Rayner, UK Deputy Prime Minister, Resig...",https://www.nytimes.com/2025/09/05/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
